# Building Interactive notebooks

As well as creating and running code we can use the @interact function to create interactive elements with very little effort. This is great for designing elements where students may need to adjust code but won't be comfortable playing with code.

#### First let's import the packages that we will need

In [ ]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interact_manual

As you can see, that didn't take very long to install all of the parts we need so you can easily add this into a class for a quick demo.

#### First we will need to create the initial singal wave that we want to interact with

In [ ]:
def signal(f1=440, f2=0, samplerate=44100, duration=0.01):
    def g(f):
        samples = np.arange(duration * samplerate) / samplerate
        return np.sin(2 * np.pi * f * samples)
    return pd.DataFrame({'s':g(f1) + g(f2)})

_=signal().plot()

And now we can create an interactive signal plot that will allow users to investigate how these waves interact.
##### Note that @interact will constantly update the plot, @interact_manual will require the user to push the change

In [ ]:
@interact
def signalplot(f1=440, f2=440):
    signal(f1, f2).plot()

In [ ]:
from IPython.display import Audio
rate =  44100

To take this a bit further we can also create an audio file that is updated from the interactive plot.

In [ ]:
@interact_manual

def audiomix(f1=400,f2=500):
    s = signal(f1=f1, f2=f2, duration=2, samplerate=rate)
    s.head(400).plot()
    display(Audio(s['s'].tolist(), rate=rate, autoplay=False))

This isn't just reserved for cool plots though, we can also build interactive elements to interact with all sorts of code. This includes being able to manipulate images.
#### Use the code below to pull in an image and build the tools we need to interact.

In [ ]:
from io import BytesIO

# Third-party libraries
from IPython.display import Image
from ipywidgets import interact, interactive, fixed
import matplotlib as mpl
from skimage import data, filters, io, img_as_float
import numpy as np

i = img_as_float(data.coffee())
i.shape


def arr2img(arr):
    """Display a 2- or 3-d numpy array as an image."""
    if arr.ndim == 2:
        format, cmap = 'png', mpl.cm.gray
    elif arr.ndim == 3:
        format, cmap = 'jpg', None
    else:
        raise ValueError("Only 2- or 3-d arrays can be displayed as images.")
    # Don't let matplotlib autoscale the color range so we can control overall luminosity
    vmax = 255 if arr.dtype == 'uint8' else 1.0
    with BytesIO() as buffer:
        mpl.image.imsave(buffer, arr, format=format, cmap=cmap, vmin=0, vmax=vmax)
        out = buffer.getvalue()
    return Image(out)


arr2img(i)

#### Now, with a little bit of effort we can create some tools to manipulate the colours and sigma of the image

In [ ]:
def edit_image(image, sigma=0.1, R=1.0, G=1.0, B=1.0):
    new_image = filters.gaussian(image, sigma=sigma, multichannel=True)
    new_image[:,:,0] = R*new_image[:,:,0]
    new_image[:,:,1] = G*new_image[:,:,1]
    new_image[:,:,2] = B*new_image[:,:,2]
    return arr2img(new_image)

In [ ]:
lims = (0.0,1.0,0.01)
interact(edit_image, image=fixed(i), sigma=(0.0,10.0,0.1), R=lims, G=lims, B=lims);

#### Finally we can take this to the next level by using this below segment to toggle the visibility of the original code, it can still be run but no longer shows. This means that you could initially allow the students to interact without scaring them away with the code. You can then toggle the code back to talk through each section.

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this Notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')